# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a classification problem, because the model of this project is to predict categories.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / (float(n_passed) + float(n_failed)) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import StratifiedShuffleSplit

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
seed = 42
#X_train, X_test, y_train, y_test 
train_index, test_index = next(iter(StratifiedShuffleSplit(y_all,test_size=num_test,random_state=seed)))
X_train, X_test = np.array(X_all)[train_index,:], np.array(X_all)[test_index,:]
y_train, y_test = y_all[train_index], y_all[test_index]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

1. Random Forest

A random forest classifier is a method that fits a number of decision tree classifiers on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting. It is easy to use, has great computational efficiency, and improves overfitting issue than regular decision trees. It is very flexible, can be used in a wide range of applications, such as credit assessment (Shi, 2011; Ghatasheh, 2014), machinery fault diagnosis (Yan, 2006; Han, 2006), face recognition (Ghoshal, 2009), prediction of aquatic toxicity (Polishchuk, 2009) and so on. There are also some weaknesses. A random forest usually is hard to provide comprehensible results, as the results are not quite interpretable. It is hard to make incremental. Whenever a new instance is added, there is no algorithm to easily add it into the model. A random forest outputs discrete results. When variables are continuous and highly linear, the model may not give a ideal output. Because this dataset has multiple discrete features, and the results are in binary categories, a random forest classifier can fit in this situation. 

2. Support Vector Machines

The advantages of SVMs are the use of kernels, the absence of local minima, the sparseness of the solution and the capacity control obtained by optimizing the margin (Shawe-Taylor, 2004). By introducing the kernels, SVMs have flexibility to separate the data, either linear or nonlinear. It has a convex optimization problem (no local minima), therefore more robust compared to the neutral network. SVMs are not good for large dataset, since the training time is scaled to be cubic in the size of the dataset. The required physical memory is also cubic scaled increasing along with data size in both training and testing. The performance of SVMs is highly depend on the selection of kernels and parameters such as C and gamma. It may runs very slow due to iteration and convergence problems. SVMs were originally developed for pattern recognition problems (Schmidt 1996; Joachims, 1998), but it has been extensively used in financial time series forecasting (Tay, 2001; Cao, 2003; Miuller, 2000; Yang, 2004 etc.). This dataset is small (only 395 samples) therefore a support vector machine can be a good candidate.

3. K-Nearest Neighbors

The cost of training is zero for a k-nearest neighbors classifier, so it is effective if the training data is large. It is also robust to noisy training data. No assumptions about the characteristic of features has to be made. The drawbacks are the expensive computational cost of prediction with large dataset, the lack of interpretability for the model, the high sensitivity to parameters (number of nearest neighbors and type of distance), and the high dependence on dimensions (curse of dimensionality) of performance. KNNs have been used in financial and economical events prediction (Imandoust, 2013), image classification (Li, 2009), pharmacogenomical analysis and prediction (Parry, 2010), text and document mining (Bijalwan, 2014) and so on. This dataset is small, and it is hard to make assumptions from the features, so k-nearest neighbors classifier can be tried.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [20]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.ensemble import RandomForestClassifier

# from sklearn import model_B
from sklearn.svm import SVC

# from skearln import model_C
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state=seed)
clf_B = SVC(random_state=seed)
clf_C = KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0302 seconds
Made predictions in 0.0036 seconds.
F1 score for training set: 0.9924.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.7368.
Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0247 seconds
Made predictions in 0.0014 seconds.
F1 score for training set: 0.9885.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.6822.
Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0284 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 0.9975.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7132.
Training a SVC using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8354.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8025.
Training a SVC using a training set size of 200. . 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Random Forest**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         .0036           |         .0015          |      .9924       |     .7368       |
| 200               |         .0014           |         .0012          |      .9885       |     .6822       |
| 300               |         .0016           |         .0012          |      .9975       |     .7132       |

** Classifer 2 - C-Support Vector (SVM)**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         .0010           |         .0010          |      .8354       |     .8025       |
| 200               |         .0034           |         .0020          |      .8431       |     .8105       |
| 300               |         .0046           |         .0015          |      .8664       |     .8052       |

** Classifer 3 - K-Nearest Neighbors**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         .0011           |         .0009          |      .8252       |     .7586       |
| 200               |         .0037           |         .0036          |      .8097       |     .7857       |
| 300               |         .0051           |         .0018          |      .8539       |     .8138       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Comparing the testing results and computational costs from three different classifier models, support vector machines will be chose as the best model. This SVM model has the highest prediction F<sub>1</sub> score among all three models. Even though it has a longer computational time than random forest classifiers, it has 10% higher F<sub>1</sub> score than random forest. This SVM has generally shorter computational time and better F<sub>1</sub> score than the k-nearest neighbors. By fine-tuning parameters, SVMs should have better performance and prediction accuracy. Considering relatively small quantity of data for this student intervention project, the computational time wouldn't be significant for this dataset. For this case, precision and accuracy are the dominant factors. In summary, SVMs are generally the most appropriate model based on available data, limited resources, cost, and performance.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

Support vector machines (SVM) are linear separators which can separator data into two categories by a straight line. For a two-dimensional case, data are plotted in a coordinate system along the x-and y-axes. SVMs will draw a straight line to separate these data into two categories. In order to keep the model validated and effective whenever new data being added into existing dataset, the separation line will be selected at the middle of two categorical boundaries where the distance to each categorical boundary is maximum. This is called maximum margin. This is so that new instances are most likely to be classified correctly. The categorical boundaries are called support lines, and they are paralleled to the separation line. Support lines are only determined by data which are on the boundaries, and only these data instances closest to the support lines matter. These data are called support vectors (see **Graph A**), from which support vector machines get the name. 

Sometimes on a two-dimensional coordinate the data are distributed like **Graph B** shown. In such case a straight line seems can’t separate data into two categories. A transformation called kernel trick can be made here. The third axis can be implemented as z = x<sup>2</sup> + y<sup>2</sup>, therefore the original two-dimensional problem is transformed to a three-dimensional problem, and all the data will be mapped into a three-dimensional space. If we look at the three-dimensional space shown as **Graph C**, in which z = x<sup>2</sup> + y<sup>2</sup> is the horizontal axis and y is the vertical axis, all the data can be separated by a straight linear line (here in three-dimensional space it is actually called a hyperplane) with maximum margin just as what has been done in two-dimensional space. Here x<sup>2</sup> + y<sup>2</sup> is our kernel function.

By using a variety of kernel functions to perform kernel tricks, some complex two-dimensional classification problems with non-linear separable data can be simplified to a much easier higher dimensional problems wth linear separable data.

![Graphs shown SVMs](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/8650/1461641606/SVM.png "SVMs")

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [21]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from numpy import logspace
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('rbf','linear','poly','sigmoid'),
              'degree':(3,4,5),
              'C':(1,1.025,1.05,1.075,1.1),
              'gamma':(0.04,0.06,0.08,0.1),
              'probability':(True,False),
              'shrinking':(True,False)}

# TODO: Initialize the classifier
clf = SVC(decision_function_shape='ovr',random_state=seed)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
cv = StratifiedShuffleSplit(y_train,test_size=0.24,random_state=seed)
grid_obj = GridSearchCV(clf,param_grid=parameters,scoring=f1_scorer,verbose=1,n_jobs=4,cv=cv)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf.get_params(), '\n'

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Fitting 10 folds for each of 960 candidates, totalling 9600 fits


[Parallel(n_jobs=4)]: Done 385 tasks      | elapsed:    7.3s
[Parallel(n_jobs=4)]: Done 1435 tasks      | elapsed:   26.2s
[Parallel(n_jobs=4)]: Done 3185 tasks      | elapsed:   57.6s
[Parallel(n_jobs=4)]: Done 5635 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 8785 tasks      | elapsed:  2.6min


{'kernel': 'rbf', 'C': 1.075, 'verbose': False, 'probability': True, 'degree': 3, 'shrinking': True, 'max_iter': -1, 'decision_function_shape': 'ovr', 'random_state': 42, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'gamma': 0.06, 'class_weight': None} 

Made predictions in 0.0054 seconds.
Tuned model has a training F1 score of 0.9437.
Made predictions in 0.0018 seconds.
Tuned model has a testing F1 score of 0.8235.


[Parallel(n_jobs=4)]: Done 9600 out of 9600 | elapsed:  2.8min finished


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F<sub>1</sub> score is 0.9437 for training, 0.8235 for testing. Both the training score and the testing score are higher than the untuned model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.